In [3]:
import rasterio
import pandas as pd
import statistics as st

In [ ]:
df_insert = pd.DataFrame({'gdd_corn': [np.NaN]*7,
                            'gdd_soy': [np.NaN]*7,
                            'gdd_wheat': [np.NaN]*7})
def dataframe_extraction(r1,r2,r3,row,col):
    """This function returns a pandas dataframe that holds the GDD values for 1-365 doys of a specific year.
    inputs: r1, r2: the GDD Rasterstacks of that specific year. Each raster has 365 layers one corresponding to a doy of that year.
            row, col: the row and column of the raster pixel we would like to get the dataframe for """
    id=np.array(range(1,1+r1.shape[0]))
    data_frame = (pd.DataFrame({'gdd_corn':r1[0:r1.shape[0]+1:1,row,col],
                                  'gdd_soy':r2[0:r1.shape[0]+1:1,row,col],
                                   'gdd_wheat':r3[0:r1.shape[0]+1:1,row,col]},index=id))
    return data_frame
    """df=pd.DataFrame()
    for i in range(len(data_frame)):
        df=pd.concat([df,data_frame.iloc[[i]],df_insert])
    df.index=range(1,1+len(df.index))
    return df[:313]"""

In [27]:
def median_from_df(l):#l is list
    l.sort()
    for i in range(len(l)):
        if l[i]!=0:
            idx=i
            break;
    q=l[idx:]
    return int(st.median(q))
def gdd_evaluate(l1,l2,temp_data,crop):
    years=list(map(str, list(range(2002,2016))))
    gdd=list()
    gdds=list()
    precip=list()
    for i in range(len(years)):
            yr=years[i]
            start_doy=l1[i]
            stop_doy=l2[i]
            t=temp_data[temp_data.Year.amin==yr]
            if(crop[i]==1):
                g=t.gdd_corn[start_doy-1:stop_doy].sum()
                gs=t.gdds_corn[start_doy-1:stop_doy].sum()
            else:
                g=t.gdd_soy[start_doy-1:stop_doy].sum()
                gs=t.gdds_soy[start_doy-1:stop_doy].sum()
            p=t.P[start_doy-1:stop_doy].sum()
            precip.append(p[0])
            gdd.append(g)
            gdds.append(gs)
            print(gs)
    print()
    return (gdd,precip)

In [38]:
usda_planting_corn=[123,129,131,123,123,127,122,134,126,122]
usda_emergence_corn=[138,138,145,138,141,141,133,144,140,136]
usda_planting_soy=[138,141,144,135,139,138,132,143,135,141]
usda_emergence_soy=[149,151,158,146,152,152,143,154,147,155]

def state_representative_phenology(crop):
    state_g=list()
    state_s=list()
    for yr in range(2006,2016,1):
        crop_phen=pd.read_csv("phenology_county_results/county_NDVI_"+str(yr)+"_"+crop+"_phenology.csv").drop("Unnamed: 0",axis=1)
        state_g.append(median_from_df(crop_phen['green'].tolist()))
        state_s.append(median_from_df(crop_phen['sen'].tolist()))
    if crop=='corn':
        df = pd.DataFrame({'usda_planting': usda_planting_corn,'usda_emergence': usda_emergence_corn,'state_greenup': state_g,'state_sen': state_s})
    elif crop=='soy':
        df = pd.DataFrame({'usda_planting': usda_planting_soy,'usda_emergence': usda_emergence_soy,'state_greenup': state_g,'state_sen': state_s})
    
    return(df)

state_representative_phenology('corn')

,state_greenup,state_sen,usda_emergence,usda_planting
0,163,256,138,123
1,164,249,138,129
2,174,271,145,131
3,163,264,138,123
4,162,252,141,123
5,174,262,141,127
6,159,245,133,122
7,170,267,144,134
8,168,265,140,126
9,165,259,136,122
